In [1]:
print('hi')

hi


In [2]:
import os

In [3]:
%pwd

'/Users/macbook/PycharmProjects/end_to_end_mlops/research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'/Users/macbook/PycharmProjects/end_to_end_mlops'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

#frozen=True: (values can’t be changed once set).
@dataclass(frozen=True)
class DataIngestionConfig:
#this class will be used for mapping the config yaml info to the python code
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [39]:
#The typicl flow is that we gonna open the yaml config files and pass the info to this dataingestion class

In [ ]:
from mlProject.constants import *
#here there are all the static path for the yaml files (config, parameters,x)

In [ ]:
from mlProject.utils.common import * #here are all teh reusable code blocks are defigned as functions and can be called and used more than once

This class is a central controller that:

Loads configuration files (config.yaml, params.yaml, schema.yaml)

Creates necessary directories

Returns structured config objects (like DataIngestionConfig) for pipeline components to use

In [35]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [36]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

This class is responsible for:

Downloading the dataset from a given URL.

Unzipping the downloaded file into a structured folder.

Logging progress and managing paths — all driven by a config object.



In [37]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [38]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-22 00:45:21,643: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-05-22 00:45:21,651: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-22 00:45:21,653: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-22 00:45:21,655: INFO: common: created directory at: artifacts]
[2025-05-22 00:45:21,656: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-22 00:45:24,234: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 3336668
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "a5376966ca8e321cbf6078b5ec05de3bb32b23e51e507c74be82d4d53ab28d1d"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7177:24F316:60FD6DD:6A77274:682E4971
Accept-Ranges: bytes
Date: Wed